In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\OneDrive\DS\GitHub_Backup\Projeto_Maximizacao_Receita
C:\Users\flavi\OneDrive\DS\GitHub_Backup\Projeto_Maximizacao_Receita


In [2]:
# Importações necessárias 

import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings

from src.config.paths import DADOS_AMOR_A_CAKES
from src.config.auxiliares_testes import analises_shapiro_levene

warnings.filterwarnings('ignore')

In [3]:
# importando da dase de dados 'dadosVenda.xlsx'
df_vendas_EDA = pd.read_excel(DADOS_AMOR_A_CAKES)
df_vendas_EDA.head()

,data,loja,produto,categoria,custo_producao,preco_original,desconto_pct,preco_final,quantidade_vendida_dia,quantidade_vendida_mes
0,2024-03-05,Tijuca,Torta de Limão,Doce,9.60,23.41,0.030,22.71,20,260
1,2024-05-22,Barra da Tijuca,Sanduíche Natural,Sanduíche,11.15,25.03,0.006,24.88,27,378
2,2024-06-22,Centro,Café Expresso,Bebida,4.35,10.18,0.004,10.14,17,408
3,2024-01-17,Barra da Tijuca,Sanduíche de Frango,Sanduíche,11.55,24.37,0.035,23.52,13,273
4,2024-04-11,Tijuca,Brigadeiro,Doce,4.12,10.14,0.037,9.76,7,161


In [7]:
df_vendas_EDA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   data                    500 non-null    object 
 1   loja                    500 non-null    object 
 2   produto                 500 non-null    object 
 3   categoria               500 non-null    object 
 4   custo_producao          500 non-null    float64
 5   preco_original          500 non-null    float64
 6   desconto_pct            500 non-null    float64
 7   preco_final             500 non-null    float64
 8   quantidade_vendida_dia  500 non-null    int64  
 9   quantidade_vendida_mes  500 non-null    int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 39.2+ KB


In [5]:
# Obeservando a estatística descritiva da base de dados 
df_vendas_EDA.describe()

,custo_producao,preco_original,desconto_pct,preco_final,quantidade_vendida_dia,quantidade_vendida_mes
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,6.376160,14.379680,0.020744,14.080580,19.734000,338.016000
std,2.851449,5.975095,0.011466,5.845572,11.152219,213.317163
min,2.010000,4.910000,0.000000,4.790000,1.000000,13.000000
25%,4.057500,9.947500,0.012000,9.732500,11.000000,161.750000
50%,5.895000,12.655000,0.022000,12.445000,19.000000,319.500000
75%,8.640000,18.995000,0.030000,18.662500,29.000000,457.000000
max,11.970000,28.810000,0.040000,28.780000,40.000000,975.000000


In [5]:
# Observando valores ausentes na base 
df_vendas.isnull().sum()

data                      0
loja                      0
produto                   0
categoria                 0
custo_producao            0
preco_original            0
desconto_pct              0
preco_final               0
quantidade_vendida_dia    0
quantidade_vendida_mes    0
dtype: int64

### Testes de Shapiro-Wilk e Levene

In [10]:
# Lista de variáveis categóricas
variaveis_categoricas = ['data', 'loja', 'produto', 'categoria']

# Remover essas colunas do DataFrame
df_numerico = df_vendas_EDA.drop(columns=variaveis_categoricas)

# Chamada da função com o DataFrame filtrado
analises_shapiro_levene(df_numerico)

Teste de Shapiro-Wilk
estatistica_sw=0.940
custo_producao não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.947
preco_original não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.958
desconto_pct não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.948
preco_final não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.957
quantidade_vendida_dia não segue uma distribuição normal (valor p: 0.000)
estatistica_sw=0.961
quantidade_vendida_mes não segue uma distribuição normal (valor p: 0.000)

Teste de Levene
estatistica_levene=917.807
Ao menos uma variância é diferente (valor p: 0.000)


---
### Observação:

- Observamos pelo teste de **Shapiro-Wilk** & **Levene** que nenhuma vaiável do dataframe **df_vendas_AED** segue uma distribuição normal. <br>
- Isso é fundamental pois, com isso, sabemos que não podemos utilizar normalização no pré-processamento.

---

### Pré-processamento 

## Conclusão da Análise para Maximização da Receita

